In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import shapely
import networkx as nx
import igraph as ig
import numpy as np

Start working with the city first, and then do the same with the metropolitan area.

In [2]:
FALLBACK_SPEED = 50
BUFFER_DUPLICATE_LS = 8
BUFFER_NEARBY = 15

In [3]:
milano_poly = gpd.read_file("../data/raw/city_boundaries/Milano.shp")
proj_crs = milano_poly.crs
proj_crs

<Projected CRS: EPSG:32632>
Name: WGS 84 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 6°E and 12°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Austria. Cameroon. Denmark. Equatorial Guinea. France. Gabon. Germany. Italy. Libya. Liechtenstein. Monaco. Netherlands. Niger. Nigeria. Norway. Sao Tome and Principe. Svalbard. Sweden. Switzerland. Tunisia. Vatican City State.
- bounds: (6.0, 0.0, 12.0, 84.0)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
milano_poly.to_crs(epsg=4326, inplace=True)

In [5]:
ox.settings.useful_tags_way += ["parking:left", "parking:right"]

In [6]:
G = ox.graph_from_polygon(milano_poly.geometry[0], network_type="drive", simplify=False)

In [7]:
G = ox.simplify_graph(
    G, edge_attrs_differ=["highway", "parking:left", "parking:right", "maxspeed"]
)

In [8]:
G = ox.add_edge_speeds(G, fallback=FALLBACK_SPEED)

In [9]:
G = ox.add_edge_travel_times(G)

In [10]:
for node in G.nodes:
    neighbors = set(list(G.predecessors(node)) + list(G.successors(node)))
    n = len(neighbors)
    d = G.degree(node)
    if node in neighbors:
        G.nodes[node]["intersection"] = True
        continue
    if G.out_degree(node) == 0 or G.in_degree(node) == 0:
        G.nodes[node]["intersection"] = True
        continue
    if not ((n == 2) and (d in {2, 4})):
        if n == 1:
            G.nodes[node]["intersection"] = False
        else:
            G.nodes[node]["intersection"] = True
        continue
    G.nodes[node]["intersection"] = False

In [11]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [12]:
ox.save_graphml(G, "../data/processed/Milan/Milan_graph_0_raw.graphml")

In [13]:
ox.save_graph_geopackage(G, "../data/processed/Milan/Milan_graph_0_raw.gpkg")

In [ ]:
amenities_dict = {
    "public_transport": ["platform"],
    "highway": [
        "crossing",
        "cyclist_waiting_aid",
        "traffic_signals",
        "street_lamp",
        "traffic_mirror",
        "bus_stop",
    ],
    "amenity": ["bicycle_parking", "parking", "marketplace"],
    "building": ["parking"],
    "place": ["square"],
    "leisure": ["park", "garden"],
    "shop": True,
    "osmid": True,
}

In [15]:
gdf = ox.features_from_polygon(
    milano_poly.geometry[0],
    tags=amenities_dict,
)

In [16]:
gdf.to_file("../data/processed/Milan/Milan_features_0_raw.gpkg")

In [17]:
def sort_values(x):
    val = []
    if x["public_transport"] == "platform" or x["highway"] == "bus_stop":
        val += ["public_transport_platform"]
    if x["leisure"] in ["park", "garden"] and x["amenity"] != "parking":
        val += ["green_area"]
    if (
        (x["place"] == "square" or x["amenity"] == "marketplace")
        and (x["leisure"] not in ["park", "garden"])
        and (x["amenity"] != "parking")
    ):
        val += ["public_square"]
    if not pd.isna(x["shop"]):
        val += ["shop"]
    if (x["amenity"] == "parking" or x["building"] == "parking") and pd.isna(x["shop"]):
        val += ["parking"]
    if x["amenity"] == "bicycle_parking":
        val += ["bicycle_parking"]
    if x["highway"] in [
        "crossing",
        "cyclist_waiting_aid",
        "traffic_signals",
        "street_lamp",
        "traffic_mirror",
    ]:
        val += [x["highway"]]
    if len(val) < 1:
        val += [None]
    return val

In [18]:
vals = []
for ind, row in gdf.iterrows():
    vals.append(sort_values(row))
gdf["type"] = vals

In [19]:
gdf["type"].value_counts()

type
[crossing]                     21611
[shop]                          8352
[public_transport_platform]     6199
[traffic_signals]               3048
[parking]                       2581
[street_lamp]                   1956
[green_area]                    1698
[bicycle_parking]                998
[public_square]                   96
[traffic_mirror]                   7
Name: count, dtype: int64

In [20]:
errors = gdf[gdf["type"].apply(lambda x: True if len(str(x).split(",")) > 1 else False)]
if len(errors) > 0:
    for i in range(len(errors)):
        print(
            errors.index[i],
            [
                val
                for val in errors.iloc[i].values
                if (
                    isinstance(val, shapely.Geometry)
                    or isinstance(val, list)
                    or isinstance(val, str)
                )
            ],
        )

In [21]:
gdf["type"] = gdf["type"].apply(lambda x: x[0])

In [22]:
gdf.to_file("../data/processed/Milan/Milan_features_1_classified.gpkg")

In [23]:
gdf_cleaned = gdf.copy()

In [24]:
gdf_po = gdf[
    gdf.geometry.apply(lambda x: True if isinstance(x, shapely.Point) else False)
]
gdf_po = gdf_po.to_crs(proj_crs)
gdf_ls = gdf[
    gdf.geometry.apply(lambda x: True if isinstance(x, shapely.LineString) else False)
]
gdf_ls = gdf_ls.to_crs(proj_crs)
gdf_ls.geometry = gdf_ls.buffer(BUFFER_DUPLICATE_LS)
duplicates = gpd.sjoin(
    gdf_ls, gdf_po, how="inner", predicate="intersects", on_attribute="type"
)
gdf_cleaned = gdf_cleaned.drop(duplicates.index.values)
print(len(gdf), len(gdf_cleaned))

46546 44063


In [25]:
gdf_cleaned.geometry = gdf_cleaned.geometry.apply(
    lambda x: x.interpolate(0.5, normalized=True)
    if isinstance(x, shapely.LineString)
    else x
)

In [26]:
gdf_cleaned.to_file("../data/processed/Milan/Milan_features_2_classified_wols.gpkg")

In [27]:
vals = []
for ind, row in gdf_cleaned.iterrows():
    if (
        isinstance(row.geometry, shapely.Polygon)
        or isinstance(row.geometry, shapely.MultiPolygon)
    ) and (row["type"] in ["shop", "public_transport_platform", "bicycle_parking"]):
        if isinstance(row.geometry, shapely.Polygon):
            vals.append(row.geometry.centroid)
        elif isinstance(row.geometry, shapely.MultiPolygon):
            vals.append(shapely.Polygon(row.geometry).centroid)
    else:
        vals.append(row.geometry)
gdf_cleaned.geometry = vals

In [28]:
gdf_cleaned[
    gdf_cleaned.geometry.apply(
        lambda x: True
        if isinstance(x, shapely.Polygon) or isinstance(x, shapely.MultiPolygon)
        else False
    )
]["type"].value_counts()

type
parking          2315
green_area       1692
public_square      71
Name: count, dtype: int64

In [29]:
gdf_cleaned[
    gdf_cleaned.geometry.apply(
        lambda x: True if isinstance(x, shapely.Point) else False
    )
]["type"].value_counts()

type
crossing                     21611
shop                          8352
public_transport_platform     3716
traffic_signals               3048
street_lamp                   1956
bicycle_parking                998
parking                        266
public_square                   25
traffic_mirror                   7
green_area                       6
Name: count, dtype: int64

In [30]:
gdf_cleaned.to_file("../data/processed/Milan/Milan_features_3_simplified.gpkg")

In [31]:
gdf_simple = gdf_cleaned[["type", "geometry"]].copy()

In [32]:
vals = []
for ind, row in gdf_simple.iterrows():
    vals.append(ind[0][0].upper() + str(ind[1]))
gdf_simple["osmid"] = vals
gdf_simple = gdf_simple.set_index("osmid")

In [33]:
gdf_simple.to_file("../data/processed/Milan/Milan_features_4_dense.gpkg")

In [34]:
gdf_simple_poly = gdf_simple[
    gdf_simple.geometry.apply(
        lambda x: True
        if isinstance(x, shapely.MultiPolygon) or isinstance(x, shapely.Polygon)
        else False
    )
]

In [35]:
gdf_simple_poly_exploded = gdf_simple_poly.explode()

In [36]:
gdf_simple_poly_exploded = gdf_simple_poly_exploded.to_crs(proj_crs)
gdf_simple_poly_exploded.geometry = gdf_simple_poly_exploded.buffer(BUFFER_NEARBY)

In [37]:
gdf_edges = gdf_edges.to_crs(proj_crs)

In [38]:
res = gpd.sjoin(gdf_edges, gdf_simple_poly_exploded, how="left", predicate="intersects")

In [39]:
grouped_res = res.groupby(["u", "v", "key"])["type"].agg(set)

In [40]:
grouped_res.values

array([{nan}, {nan}, {'green_area', 'parking'}, ..., {'parking'}, {nan},
       {nan}], shape=(26388,), dtype=object)

In [41]:
gdf_edges["near_parking"] = [
    True if "parking" in x else False for x in grouped_res.values
]
gdf_edges["near_park"] = [
    True if "green_area" in x else False for x in grouped_res.values
]
gdf_edges["near_square"] = [
    True if "public_square" in x else False for x in grouped_res.values
]

In [42]:
left_parking = [
    True if (not pd.isna(val) and val != "no") else False
    for val in gdf_edges["parking:left"].values
]
right_parking = [
    True if (not pd.isna(val) and val != "no") else False
    for val in gdf_edges["parking:right"].values
]

In [43]:
gdf_edges["street_parking"] = [
    left or right for left, right in zip(left_parking, right_parking)
]

In [44]:
highway_dict = {
    "motorway": 1,
    "trunk": 2,
    "primary": 3,
    "secondary": 4,
    "tertiary": 5,
    "unclassified": 6,
    "residential": 7,
    "living_street": 7,
    "busway": 8,
    "emergency_bay": 8,
    "road": 8,
}

gdf_edges["hierarchy"] = gdf_edges["highway"]
gdf_edges["hierarchy"] = gdf_edges["hierarchy"].apply(lambda x: x.removesuffix("_link"))
gdf_edges["hierarchy"] = gdf_edges["hierarchy"].apply(lambda x: highway_dict[x])

In [45]:
gdf_edges["speed_kph"] = gdf_edges["speed_kph"].map(lambda x: int(round(x, -1)))

In [46]:
gdf_edges = gdf_edges.to_crs(epsg=4326)

In [47]:
gdf_nodes["traffic_signals"] = [
    True if (isinstance(val, str) and "traffic_signals" in val) else False
    for val in gdf_nodes["highway"].values
]

In [48]:
G = ox.graph_from_gdfs(gdf_nodes=gdf_nodes, gdf_edges=gdf_edges, graph_attrs=G.graph)

In [49]:
ox.save_graphml(G, "../data/processed/Milan/Milan_graph_1_all.graphml")
ox.save_graph_geopackage(G, "../data/processed/Milan/Milan_graph_1_all.gpkg")

In [50]:
gdf_edges

osmid      highway lanes  \
u           v           key                                              
10371529    743371634   0                 274433644      primary     2   
            743371622   0                  26703159     tertiary   NaN   
10371530    2754579083  0    [270390234, 270390223]      primary     2   
13595397    271096577   0                 195654896    secondary     2   
            344814479   0                 195654901    secondary   NaN   
...                                             ...          ...   ...   
13229923006 258161549   0                 264287860    secondary     2   
13229923008 2645692103  0                 246295275    secondary     2   
13230320604 2516127373  0                  23634440  residential   NaN   
            13230320605 0    [266430890, 502202854]    secondary     2   
13230320605 256039870   0                 266430890    secondary   NaN   

                                              name  oneway reversed  \
u           v           key                                           
10371529    743371634   0               Via Novara    True    False   
            743371622   0               Via Novara    True    False   
10371530    2754579083  0               Via Novara    True    False   
13595397    271096577   0            Viale Certosa   False    False   
            344814479   0            Viale Certosa    True    False   
...                                            ...     ...      ...   
13229923006 258161549   0         Viale Gran Sasso    True    False   
13229923008 2645692103  0    Piazzale Gabrio Piola    True    False   
13230320604 2516127373  0     Via Antonio Raimondi    True    False   
            13230320605 0     Via Console Marcello    True    False   
13230320605 256039870   0     Via Console Marcello    True    False   

                                 length  speed_kph  travel_time  \
u           v           key                                       
10371529    743371634   0    154.435012         50    10.992938   
            743371622   0    150.466270         40    12.127412   
10371530    2754579083  0    385.419906         70    19.821595   
13595397    271096577   0     41.927380         50     3.018771   
            344814479   0     16.212785         50     1.203935   
...                                 ...        ...          ...   
13229923006 258161549   0     49.282513         50     3.548341   
13229923008 2645692103  0     35.052402         50     2.523773   
13230320604 2516127373  0     20.181762         40     1.991974   
            13230320605 0     20.734411         50     1.492878   
13230320605 256039870   0     62.696707         50     4.514163   

                                                                      geometry  \
u           v           key                                                      
10371529    743371634   0      LINESTRING (9.07662 45.48756, 9.07469 45.48785)   
            743371622   0    LINESTRING (9.07662 45.48756, 9.07611 45.48771...   
10371530    2754579083  0    LINESTRING (9.10205 45.47981, 9.1017 45.47991,...   
13595397    271096577   0      LINESTRING (9.13066 45.49953, 9.13021 45.49973)   
            344814479   0      LINESTRING (9.13066 45.49953, 9.13066 45.49967)   
...                                                                        ...   
13229923006 258161549   0    LINESTRING (9.22323 45.48076, 9.22301 45.48085...   
13229923008 2645692103  0    LINESTRING (9.22301 45.4806, 9.22294 45.48051,...   
13230320604 2516127373  0    LINESTRING (9.14179 45.50336, 9.14178 45.5033,...   
            13230320605 0    LINESTRING (9.14179 45.50336, 9.14188 45.50328...   
13230320605 256039870   0    LINESTRING (9.14195 45.50321, 9.14215 45.50303...   

                             ... bridge tunnel width access est_width  \
u           v           key  ...                                        
10371529    743371634   0    ...    NaN    NaN   NaN    NaN       NaN   
 

In [51]:
gdf_edges_simplified = gdf_edges.drop(
    [
        "lanes",
        "junction",
        "ref",
        "bridge",
        "tunnel",
        "width",
        "access",
        "est_width",
        "reversed",
        "parking:right",
        "parking:left",
        "maxspeed",
    ],
    axis=1,
)

In [52]:
gdf_nodes_simplified = gdf_nodes.drop(["highway", "ref", "junction", "railway"], axis=1)

In [53]:
G = ox.graph_from_gdfs(
    gdf_nodes=gdf_nodes_simplified, gdf_edges=gdf_edges_simplified, graph_attrs=G.graph
)

In [54]:
ox.save_graphml(G, "../data/processed/Milan/Milan_graph_2_dense.graphml")
ox.save_graph_geopackage(G, "../data/processed/Milan/Milan_graph_2_dense.gpkg")

In [55]:
G_ig = ig.Graph.from_networkx(G)

In [56]:
bet_length = G_ig.edge_betweenness(directed=True, weights="length")

In [57]:
G_ig.es["edge_betweenness_centrality_length"] = np.array(bet_length) / (
    len(G.edges) * (len(G.edges) - 1)
)

In [58]:
bet_time = G_ig.edge_betweenness(directed=True, weights="travel_time")

In [59]:
G_ig.es["edge_betweenness_centrality_time"] = np.array(bet_time) / (
    len(G.edges) * (len(G.edges) - 1)
)

In [60]:
H = G_ig.to_networkx()

In [61]:
ox.save_graphml(H, "../data/processed/Milan/Milan_graph_3_metrics.graphml")
ox.save_graph_geopackage(H, "../data/processed/Milan/Milan_graph_3_metrics.gpkg")

In [62]:
hnodes, hedges = ox.graph_to_gdfs(H)

In [63]:
gdf_edges_simplified = hedges.copy()
gdf_edges_simplified["osmid"] = hedges["osmid"].apply(
    lambda x: "W" + str(x) if not isinstance(x, list) else ["W" + str(val) for val in x]
)
gdf_edges_simplified = hedges.set_index(keys="osmid")
gdf_edges_simplified = gdf_edges_simplified.drop("_igraph_index", axis=1)

In [64]:
gdf_nodes_simplified = hnodes.copy()
gdf_nodes_simplified.index = ["N" + str(x) for x in gdf_nodes_simplified.index]
gdf_nodes_simplified = gdf_nodes_simplified.drop("_igraph_index", axis=1)

In [65]:
gdf_edges_simplified["origin"] = "road"
gdf_nodes_simplified["origin"] = "node"
gdf_simple["origin"] = "features"

In [66]:
nf = gpd.sjoin(
    gdf_simple[gdf_simple.geometry.apply(lambda x: True if isinstance(x, shapely.Point) else False)],
    gdf_nodes_simplified,
    how="left",
    predicate="intersects"
    )

In [67]:
duplicates = nf[pd.notna(nf["origin_right"])].index.values

In [74]:
gdf_nodes_simplified_curated = gdf_nodes_simplified.drop(duplicates, axis=0)

In [75]:
gdf_nodes_simplified_curated = gdf_nodes_simplified_curated[gdf_nodes_simplified_curated["intersection"].values]
gdf_nodes_simplified_curated["type"] = "intersection"

In [76]:
gdf_simple_curated = gdf_simple.copy()
gdf_simple_curated["origin"] = [
    [row["origin"], "node"] if ind in duplicates else row["origin"]
    for ind, row in gdf_simple_curated.iterrows()
]
gdf_simple_curated = gdf_simple_curated.drop_duplicates(
    subset=["geometry", "type"], keep="first"
)

In [77]:
joined = pd.concat(
    [gdf_simple_curated, gdf_edges_simplified, gdf_nodes_simplified_curated]
)

In [78]:
gdf_edges_simplified

,name,speed_kph,near_square,hierarchy,geometry,near_park,length,travel_time,oneway,near_parking,highway,street_parking,edge_betweenness_centrality_length,edge_betweenness_centrality_time,origin
osmid,,,,,,,,,,,,,,,
274433644,Via Novara,50,False,3,"LINESTRING (9.07662 45.48756, 9.07469 45.48785)",False,154.435012,10.992938,True,False,primary,False,0.000831,0.001288,road
26703159,Via Novara,40,False,5,"LINESTRING (9.07662 45.48756, 9.07611 45.48771...",False,150.466270,12.127412,True,False,tertiary,False,0.001023,0.000478,road
274433661,Via Novara,50,False,3,"LINESTRING (9.07469 45.48785, 9.07426 45.48793)",False,34.101035,2.427368,True,False,primary,False,0.000834,0.001297,road
59995455,Via Novara,40,False,5,"LINESTRING (9.07469 45.48785, 9.07482 45.48791...",False,47.848772,3.856557,True,False,tertiary,False,0.000006,0.000009,road
59995448,Via Novara,40,False,5,"LINESTRING (9.07492 45.4882, 9.07463 45.48832)",False,26.992731,2.175584,True,False,tertiary,False,0.001030,0.000487,road
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55178931,Piazzale Cimitero Maggiore,50,False,4,"LINESTRING (9.12201 45.50358, 9.12208 45.50371...",False,69.014339,5.124893,True,False,secondary,False,0.000148,0.000661,road
1418617031,None,50,False,4,"LINESTRING (9.12201 45.50358, 9.12199 45.5036,...",True,10.432183,0.774677,True,False,secondary,False,0.000091,0.000093,road
55178931,Piazzale Cimitero Maggiore,50,False,4,"LINESTRING (9.12245 45.50412, 9.12243 45.50415...",True,11.258415,0.836032,True,False,secondary,False,0.000069,0.000632,road


In [79]:
gdf_simple

,type,geometry,origin
osmid,,,
N13919635,crossing,POINT (9.16073 45.4877),features
N13919640,traffic_signals,POINT (9.15375 45.48613),features
N13919655,crossing,POINT (9.16219 45.48767),features
N13919747,traffic_signals,POINT (9.1608 45.4808),features
N13919750,crossing,POINT (9.16551 45.48758),features
...,...,...,...
W1437085224,parking,"POLYGON ((9.20911 45.40497, 9.20931 45.40501, ...",features
W1437085225,parking,"POLYGON ((9.20934 45.40499, 9.20954 45.40503, ...",features
W1437085226,parking,"POLYGON ((9.20956 45.40505, 9.20974 45.40508, ...",features


In [80]:
joined.to_file("../data/processed/Milan/Milan_all.gpkg")